# **Feature Extraction Models**

In [1]:
import pandas as pd
import nltk
import gensim
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from gensim.parsing.preprocessing import STOPWORDS
import numpy as np
from gensim import corpora,models
import time
import pickle
nltk.download('stopwords')
nltk.download('wordnet')
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\niranjans3ln\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\niranjans3ln\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# open a file, where you stored the pickled data
f = open('dataset/bow_corpus.pickle', 'rb')
bow_corpus=pickle.load(f)

file = open('dataset/dictionary.pickle', 'rb')
dictionary=pickle.load(file)

# later on, load trained model from file
lda_model =  models.LdaModel.load('dataset/lda_model.model')

In [3]:
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 5:
            result.append(lemmatize(token))
    return result

In [5]:
#Import all the clusters from the drive 
for c in range(10):
  exec('topic_{} = pd.read_csv("dataset/topic_{}.csv")'.format(c,c))
  exec("topic_{}= topic_{}.drop(columns=['Unnamed: 0'])".format(c,c))
  exec("topic_{}['Description'] = topic_{}['Description'].map(preprocess)".format(c,c))

In [6]:
#creating a corpus for Word2Vec and FastText models
for i in range(10):
  exec('sent_{} = []'.format(i))
  exec('x= topic_{}'.format(i))
  for j in range(len(x)):
    exec('sent_{}.append(topic_{}.Description[{}])'.format(i,i,j))

In [7]:
#Length of all the corpus
for sent in range(10):
  exec('print(len(sent_{}))'.format(sent))

126
104
114
123
94
111
49
79
94
142


In [10]:
#!pip install glove_python

  ERROR: Command errored out with exit status 1:
   command: 'C:\Anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\niranjans3ln\\AppData\\Local\\Temp\\pip-install-ydr7ifc7\\glove-python_967621288cb94b3389cdbf0335aa4ca8\\setup.py'"'"'; __file__='"'"'C:\\Users\\niranjans3ln\\AppData\\Local\\Temp\\pip-install-ydr7ifc7\\glove-python_967621288cb94b3389cdbf0335aa4ca8\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\niranjans3ln\AppData\Local\Temp\pip-wheel-qzmslhgh'
       cwd: C:\Users\niranjans3ln\AppData\Local\Temp\pip-install-ydr7ifc7\glove-python_967621288cb94b3389cdbf0335aa4ca8\
  Complete output (14 lines):
  C:\Anaconda3\lib\site-packages\setuptools\dist.py:757: UserWarning: Usage of dash


  Running setup.py clean for glove-python
Failed to build glove-python
    Running setup.py install for glove-python: started
    Running setup.py install for glove-python: finished with status 'error'



    error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
    ----------------------------------------
ERROR: Command errored out with exit status 1: 'C:\Anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\niranjans3ln\\AppData\\Local\\Temp\\pip-install-ydr7ifc7\\glove-python_967621288cb94b3389cdbf0335aa4ca8\\setup.py'"'"'; __file__='"'"'C:\\Users\\niranjans3ln\\AppData\\Local\\Temp\\pip-install-ydr7ifc7\\glove-python_967621288cb94b3389cdbf0335aa4ca8\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record 'C:\Users\niranjans3ln\AppData\Local\Temp\pip-record-hcior3be\install-record.txt' --single-version-ex

In [9]:
from gensim.models import Word2Vec,FastText
from gensim.test.utils import get_tmpfile
from glove import Glove, Corpus

ModuleNotFoundError: No module named 'glove'

### **Word2Vec model training**

In [ ]:
for cluster in range(10):
  #Preparing parameters for model training
  exec('corpus = sent_{}'.format(cluster))
  vector_size = 100
  w = 6
  min_count = 5
  CBoW = 0
  iterations = 100

  #Training Word2Vec model for each cluster
  exec ('w2vmodel{} = Word2Vec(corpus, size=vector_size, window=w, min_count=min_count, sg=CBoW, iter=iterations)'.format(cluster))

  #Save the all the models in individual file
  exec('path = get_tmpfile("dataset/word2vec{}.model")'.format(cluster))
  exec('w2vmodel{}.save("dataset/word2vec{}.model")'.format(cluster), cluster))

### **FastText model training**

In [ ]:
for cluster in range(10):
  #Preparing parameters for model training
  exec('corpus = sent_{}'.format(cluster))
  vector_size = 100
  w = 6
  min_count = 5
  CBoW = 0
  iterations = 100

  #Training FastText model for each cluster
  exec ('ftmodel{} = FastText(corpus, size=vector_size, window=w, min_count=min_count, sg=CBoW, iter=iterations)'.format(cluster))

  #Save the all the models in individual file
  exec('path = get_tmpfile("dataset/ftmodel{}.model")'.format(cluster))
  exec('ftmodel{}.save("dataset/ftmodel{}.model")'.format(cluster, cluster))

### **GloVe model training**

In [ ]:
#Training GloVe model for each cluster
for cluster in range(10):
  vector_size = 100
  exec('glove_corpus{}=Corpus()'.format(cluster, cluster)) 
  exec('glove_corpus{}.fit(sent_{})'.format(cluster, cluster))
  exec('glove{}= Glove(no_components=vector_size, learning_rate=0.18, alpha=0.75, max_count=100, max_loss=10.0, random_state=None)'.format(cluster, cluster))
  exec('glove{}.fit(glove_corpus{}.matrix, epochs=1000, no_threads=3, verbose=True)'.format(cluster, cluster))
  exec('transformer = lambda dictionary2:glove{}.transform_paragraph(words, epochs=1000,ignore_missing=False)'.format(cluster, cluster))
  exec('glove{}.add_dictionary(glove_corpus{}.dictionary)'.format(cluster, cluster))

  #Save the all the models in individual file
  exec('path = get_tmpfile("dataset/glove{}.model")'.format(cluster))
  exec('glove{}.save("dataset/glove{}.model")'.format(cluster, cluster))